# Speech Denoising & Classification

## Mounting google drive to colab

In [0]:
# Comment the line if necessary for deployment
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Checking whether mount is successful

In [0]:
# Comment the line if necessary for deployment
!ls "drive/My Drive"

 Classroom	   'Getting started.pdf'	  Project
'Colab Notebooks'  'NAAMII Winter School in AI'   voice1.wav


## Importing necessary Libraries

In [0]:
import numpy as np

import scipy.signal as ss

from keras.models import load_model

Using TensorFlow backend.


## Initialize global variables

In [0]:
windowLength = 256
win          = np.hamming(windowLength)
overlap      = round(0.75 * windowLength)
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8

In [0]:
model_clf = load_model("drive/My Drive/Project/Models/speech_vs_non_speech_cnn_model_with_param_14,418_val_loss_0.0032_val_acc_0.9956_tf_version_old.h5")
#model = load_model("drive/My Drive/Project/Models/cnn_model_param_1,023,381_val_loss_6607.3393_val_acc_0.1635.h5")

W0730 08:24:38.355627 140630898108288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0730 08:24:38.376812 140630898108288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



## Method to get 8 segment of stft

In [0]:
def get_8segments_xmag(stft_datastore, num_features=129, num_timestamps=510, num_segments=8):
    
    stft_datastore = stft_datastore.reshape(-1,num_features,num_timestamps)
        
    data_stft_segments = []
        
    for stft in stft_datastore:
      
        data_stft = stft
        data_stft = np.hstack((data_stft[:,0:num_segments - 1], data_stft))
        
        stft_segments = np.zeros((num_segments, num_features, data_stft.shape[1] - num_segments + 1))
        
        for index in (range(0,data_stft.shape[1] - num_segments + 1)):
            stft_segments[:,:,index] = data_stft[:,index:index + num_segments].T
        
        for i in range(0,num_timestamps, 17):
          data_stft_segments.append(stft_segments[:,:,i:i+17])
        
    data_stft_segments = np.array(data_stft_segments)
    
 
            
    return data_stft_segments

## Function to get 1 segment of STFT

In [0]:
def get_1segment_ymag(stft_datastore,num_features=129, num_timestamps=510):
    stft_datastore = stft_datastore.reshape(-1,num_features,num_timestamps)
        
    data_stft_segments = []
    
    #data_stft_segments = [stft[:,index:index+17] for stft in tqdm(stft_datastore) for index in range(0,num_timestamps,17)]
        
    for stft in stft_datastore:
        for index in range(0,num_timestamps,17):
            
            data_stft_segments.append(stft[:,index:index+17])

    data_stft_segments = np.array(data_stft_segments)
    data_stft_segments = data_stft_segments.reshape(data_stft_segments.shape[0],1,num_features, 17)
    
    return data_stft_segments

## Function to find whether the signal is speech or not

In [0]:
def is_speech(mag):
  #mag = mag[0]
  return True if (np.argmax(model_clf.predict(mag)) == 0) else False


## Function to get clean speech

In [0]:
def get_clean_speech(audio_data):
  #power = np.sum(audio_data.astype('int32')**2)
  stft = ss.stft(audio_data,fs=fs,window='hamm',noverlap=overlap,nfft=ffTLength)[2]
  mag = np.abs(stft)
  phase = np.angle(stft)
  #print(mag.shape)
  #stft = get_8segments_xmag(mag, num_features=129, num_timestamps=17, num_segments=8)
  #phase = get_1segment_ymag(phase,num_features=129, num_timestamps=510)
  #print(stft.shape)
  #mag = model.predict(stft)
  #print(mag.shape)
  #print(is_speech(mag))
  #print(power)
  #if (power>50000000) and is_speech(mag):
  if is_speech(mag.reshape(1,1,129,17)):
    #stft = mag[0] * np.exp(1j*phase[0])
    #audio_data = ss.istft(stft,fs=fs,window='hamm',noverlap=overlap,nfft=ffTLength)
    #audio_data = audio_data[1].astype('int16')
    return True#, audio_data

  else:
    return False#, False
